In [1]:
import os
import tensorflow as tf

In [2]:
#Extractict the zip file.
zip_file = tf.keras.utils.get_file(origin="E:\TF_Flower\fp.zip", fname = "fp.zip", extract=True , cache_subdir="E:\TF_Flower")


In [3]:
base_dir = os.path.join(os.path.dirname(zip_file), "fp")

In [4]:
IMAGE_SIZE = 244
BATCH_SIZE = 128

#PreProcessing the data.
data = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

#Train Data
train_generator = data.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset="training"
)

#Validation Data
val_generator = data.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset="validation"
)

Found 5935 images belonging to 102 classes.
Found 1435 images belonging to 102 classes.


In [5]:
print(train_generator.class_indices)

{'Alpine sea holly': 0, 'Anthurium': 1, 'Artichoke': 2, 'Azalea': 3, 'Ball moss': 4, 'Balloon flower': 5, 'Barbeton daisy': 6, 'Bearded iris': 7, 'Bee balm': 8, 'Bird of paradise': 9, 'Bishop of llandaff': 10, 'Black-eyed susan': 11, 'Blackberry lily': 12, 'Blanket flower': 13, 'Bolero deep blue': 14, 'Bougainvillea': 15, 'Bromelia': 16, 'Buttercup': 17, 'Californian poppy': 18, 'Camellia': 19, 'Canna lily': 20, 'Canterbury bells': 21, 'Cape flower': 22, 'Carnation': 23, 'Cautleya spicata': 24, 'Clematis': 25, "Colt's foot": 26, 'Columbine': 27, 'Common dandelion': 28, 'Corn poppy': 29, 'Cyclamen': 30, 'Daffodil': 31, 'Desert-rose': 32, 'English marigold': 33, 'Fire lily': 34, 'Foxglove': 35, 'Frangipani': 36, 'Fritillary': 37, 'Garden phlox': 38, 'Gaura': 39, 'Gazania': 40, 'Geranium': 41, 'Giant white arum lily': 42, 'Globe thistle': 43, 'Globe-flower': 44, 'Grape hyacinth': 45, 'Great masterwort': 46, 'Hard-leaved pocket orchid': 47, 'Hibiscus': 48, 'Hippeastrum': 49, 'Japanese anem

In [6]:
labels = '\n'.join(sorted(train_generator.class_indices.keys()))


"""
w : Opens in write-only mode. The pointer is placed at the beginning of the file and this will overwrite any existing file with the same name.
It will create a new file if one with the same name doesn't exist.

"""

with open('labels.txt', 'w') as f:
    f.write(labels)

In [7]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

##MobileNetV2 :- MobileNetV2 is a significant improvement over MobileNetV1 and pushes the state of the art for mobile visual recognition including classification, object detection and semantic segmentation.

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

In [8]:
#freezing the model
base_model.trainable=False

In [9]:
#Layers
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(102, activation='softmax')
])

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
epochs = 20

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)

Epoch 1/20
47/47 [==============================] - 613s 13s/step - loss: 4.3555 - accuracy: 0.0739 - val_loss: 3.2603 - val_accuracy: 0.2892
Epoch 2/20
47/47 [==============================] - 325s 7s/step - loss: 2.8513 - accuracy: 0.3684 - val_loss: 2.1010 - val_accuracy: 0.4983
Epoch 3/20
47/47 [==============================] - 312s 7s/step - loss: 1.6770 - accuracy: 0.5932 - val_loss: 1.5687 - val_accuracy: 0.5958
Epoch 4/20
47/47 [==============================] - 1867s 40s/step - loss: 1.0434 - accuracy: 0.7394 - val_loss: 1.2730 - val_accuracy: 0.6683
Epoch 5/20
47/47 [==============================] - 326s 7s/step - loss: 0.8050 - accuracy: 0.7870 - val_loss: 1.1863 - val_accuracy: 0.6927
Epoch 6/20
47/47 [==============================] - 316s 7s/step - loss: 0.6447 - accuracy: 0.8397 - val_loss: 1.0516 - val_accuracy: 0.7220
Epoch 7/20
47/47 [==============================] - 315s 7s/step - loss: 0.5208 - accuracy: 0.8738 - val_loss: 1.0140 - val_accuracy: 0.7352
Epoch 8/20

In [12]:
saved_model_dir = ""
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()


"""
wb : Opens a write-only file in binary mode.
"""

with open('modelflow.tflite','wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: assets
